# Introduction

See "demo.ipynb" notebook for a common introduction.

This notebook demonstrates how to use the spectral fabric model by Rathmann et al. (2021) and Rathmann and Lilien (2021) to construct a vertical profile of the crystal Orientation Distribution Function (ODF), needed to calculate the second-order structure tensor,
$$\langle\mathbf{c}\otimes\mathbf{c}\rangle = \int_{S^2} \mathbf{c}\otimes\mathbf{c} \,\psi \, \mathrm{d}\Omega,$$
for each layer in the radar model.

# Synthetic fabric profile

The fabric model accounts for the ODF evolution when subject to kinematic lattice (c-axis) rotation (Svendsen and Hutter, 1996) and discontinuous dynamic recrystallization (DDRX) (Placidi and others, 2010):

$$ \frac{\mathrm{D} \psi}{\mathrm{D} t} + \nabla_{S^2}\cdot(\psi\mathbf{\dot{c}}) = \Gamma_0(D-\langle D \rangle) \psi + \nu\nabla^2\psi.$$

The parameters $\Gamma_0$ and $\nu$ specify the rate of DDRX and the strength of regularization, respectively. 
$\mathbf{\dot{c}}$ is the c-axis velocity field that depends on the macroscopic strain-rate and spin tensors, while $D$ is the single-grain deformability (normalized resolved shear stress squared) that depends on the deviatoric stress tensor (see Rathmann et al. (2021) and Rathmann and Lilien (2021) for details).

Vertical fabric profiles are synthesized by linearly transposing the time-scale of an ice-parcel deformation experiment onto a depth scale. 
Two consecutive modes of deformation must be specified per deformation experiment via the structures $\verb|mode1|$ and $\verb|mode2|$ below.

**Try out these three example profiles to get started:**

In [ ]:
import numpy as np
from synthfabric import * # Requires specfabpy: pip3 install specfabpy

H        = 2000   # Ice column thickness
N_layers = 100    # Number of layers (for radar model below)
L        = 10     # Spectral truncation
nu       = 2.5e-3 # Regularization magnitude (to be adjusted depending on "L" and the time-scale "t_c" specified below)

synfab = SyntheticFabric(H=H, N=N_layers, L=L, nu=nu) 

"""
Specify the two modes of deformation by setting the fields of "mode1" and "mode2":


*** Pure shear: 
    ax        The compressional/extensional axis ("x", "y" or "z").
    t_c       Time (years) taken to halve the parcel height if t_c>0, or double if t_c<0.
    r=[-1;1]  Pure shear confinement: r=0 => unconfined, r=1 or r=-1 => 100% confinement in either of the non-"ax" directions.
    Gamma0    DDRX rate factor

*** Simple shear:
    plane     Shear plane ("zx" or "yx")
    t_c       Time (years) taken to reach a shear strain of 1
    Gamma0    DDRX rate factor

------

Calling synfab.makeProfile() requires passing:

    dt               Integration time step (years)
    t_end            Total integration time (years)
    crossover=[a,b]  Relative points in time over which the two deformational modes are linearly transitioned between.
    
"""

# Example 1
#mode1 = {   'ax':'z',  't_c':500, 'r':+1, 'Gamma0':0}
#mode2 = {'plane':'zx', 't_c':200,         'Gamma0':0}
#nlm, lm, z = synfab.make_profile(mode1, mode2, dt=10, t_end=1000, crossover=[0.5,0.6], plot=True) 

# Example 2
mode1 = {   'ax':'x',  't_c':-500, 'r':0, 'Gamma0':0}
mode2 = {'plane':'zx', 't_c':200,        'Gamma0':0}
nlm, lm, z = synfab.make_profile(mode1, mode2, dt=10, t_end=1000, crossover=[0.5,0.6], plot=True) 

# Example 3
#mode1 = {'plane':'yx', 't_c':500, 'Gamma0':0}
#mode2 = {'plane':'zx', 't_c':500, 'Gamma0':1e-9}
#nlm, lm, z = synfab.make_profile(mode1, mode2, dt=10, t_end=1000, crossover=[0.5,0.7], plot=True) 

# Calculate radar returns

To calculate the radar returns 

* $\bar{P}_{jk}$          : Mean return power for Tx and Rx polarizations of $j=\lbrace \mathrm{H,V}\rbrace$ and $k=\lbrace \mathrm{H,V}\rbrace$, respectively
* $\delta{P}_{jk}$        : Angular power anomaly for Tx and Rx polarizations of $j=\lbrace \mathrm{H,V}\rbrace$ and $k=\lbrace \mathrm{H,V}\rbrace$, respectively
* $\varphi_{\mathrm{HV}}$ : H$-$V Covariance phase angle

given the above fabric profile, run:


In [ ]:
import sys
sys.path.insert(0, '../../lib') # Add model library path
from layerstack import *
from plottools import *

### Setup layer stack

N_frames = 50           # Number of horizontally rotated frames (beta) between 0 and Pi 
epsa     = 3.17         # Single grain permittivity perpendicular to c-axis
epsc     = epsa - 0.034 # Single grain permittivity parallel to c-axis
sigma    = 1e-5         # Isotropic macroscopic conductivity

lstack = LayerStack(nlm, z, N_frames=N_frames, epsa=[epsa], epsc=[epsc], sigma=[sigma]) # Assumes constant epsa, epsc and sigma throughout the column if len()==1

### Transmitted plane wave

E0       = 1e3   # Tx E-field amplitude
f        = 179e6 # Tx frequency in Hz
alpha    = np.deg2rad(0) # Angle of incidence in radians (0 = normal incidence)

returns = lstack.get_returns(E0, f=f, alpha=alpha) # returns = (Pm_HH,Pm_HV, dP_HH,dP_HV, c_HHVV, E_HH,E_HV)

### Plot results

(eigvals, e1,e2,e3, a2) = lstack.get_eigenbasis()

(plt, *_) = plot_returns(lstack.z, returns, a2, eigvals) # NOTE: a2 contains only information about the lowest order harmonic modes. The ODF plots will therefore NOT look like the above plotted ODFs.
plt.show()